In [2]:
import polars as pl

# Load parquet files from GCS path
df = pl.read_parquet(
    "gs://zenml-472221-zenml-artifacts/ml_forecast/preprocess_data/training_data/b84e71d0-8024-4924-ac03-42f0ba96a323/a111eb87/1/8/df.parquet.gzip"
)

In [3]:
df

ds,y,__index_level_0__
datetime[ns],i64,i64
2024-01-01 00:00:00,13,8
2024-01-02 00:00:00,5,23
2024-01-03 00:00:00,5,38
2024-01-04 00:00:00,9,53
2024-01-05 00:00:00,8,68
…,…,…
2024-03-08 00:00:00,8,1013
2024-03-09 00:00:00,16,1028
2024-03-10 00:00:00,26,1043
